# CSIRO Image2Biomass - Hydra Vision Inference

This notebook uses the **Hydra Architecture** (1 Backbone, 5 Specialized Heads). 
It is optimized for a scenario where **metadata is missing** from the test set. 
The model has been trained locally using a consistency loss to ensure physical biomass constraints.

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
DATA_DIR = r"d:\personalProject\CSIRO-Image2Biomass_Prediction\csiro-biomass"
CHECKPOINT_PATH = "../models_checkpoints/best_hydra_vision_fold1.pth"
IMAGE_SIZE = (384, 384)
TARGET_COLUMNS = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'GDM_g', 'Dry_Total_g']

In [ ]:
class HydraVisionModel(nn.Module):
    def __init__(self, model_name='convnext_tiny.in12k_ft_in1k', pretrained=False):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0)
        embed_dim = self.backbone.num_features
        
        self.head_clover = nn.Sequential(nn.Linear(embed_dim, 256), nn.GELU(), nn.Linear(256, 1))
        self.head_dead   = nn.Sequential(nn.Linear(embed_dim, 256), nn.GELU(), nn.Linear(256, 1))
        self.head_green  = nn.Sequential(nn.Linear(embed_dim, 256), nn.GELU(), nn.Linear(256, 1))
        self.head_gdm    = nn.Sequential(nn.Linear(embed_dim, 256), nn.GELU(), nn.Linear(256, 1))
        self.head_total  = nn.Sequential(nn.Linear(embed_dim, 256), nn.GELU(), nn.Linear(256, 1))
        
    def forward(self, image):
        features = self.backbone(image)
        clover = self.head_clover(features)
        dead = self.head_dead(features)
        green = self.head_green(features)
        gdm = self.head_gdm(features)
        total = self.head_total(features)
        return torch.cat([clover, dead, green, gdm, total], dim=1)

class SimpleDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = np.array(Image.open(os.path.join(self.img_dir, row['image_path'])).convert('RGB'))
        if self.transform:
            image = self.transform(image=image)['image']
        return image

In [ ]:
def run_inference():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
    unique_test = test_df.drop_duplicates(subset=['image_path']).copy()
    
    model = HydraVisionModel().to(device)
    if os.path.exists(CHECKPOINT_PATH):
        model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=device))
    model.eval()
    
    transform = A.Compose([
        A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1]),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    
    dataset = SimpleDataset(unique_test, DATA_DIR, transform)
    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    
    results = []
    with torch.no_grad():
        for i, image in enumerate(loader):
            image = image.to(device)
            preds = model(image).cpu().numpy()[0]
            img_path = unique_test.iloc[i]['image_path']
            for j, col in enumerate(TARGET_COLUMNS):
                results.append({
                    'image_path': img_path,
                    'target_name': col,
                    'target': max(0, float(preds[j]))
                })
    
    pred_df = pd.DataFrame(results)
    submission = test_df[['sample_id', 'image_path', 'target_name']].merge(pred_df, on=['image_path', 'target_name'], how='left')
    submission = submission[['sample_id', 'target']]
    submission.to_csv("submission_hydra_vision.csv", index=False)
    print("Submission saved to submission_hydra_vision.csv")

if __name__ == "__main__":
    run_inference()